In [1]:
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import gc
import os
from torchga.torchga import GeometricAlgebra
import torch.nn as nn
import pyro
import pyro.infer.mcmc as mcmc
from pyro.infer.mcmc import HMC, MCMC

In [2]:
if torch.cuda.is_available():
    # Set memory growth behavior (manually or automatically managed by CUDA)
    for i in range(torch.cuda.device_count()):
        # Set the memory fraction (optional, defaults to 1.0 meaning use all available memory)
        torch.cuda.set_per_process_memory_fraction(1.0, i)
        
        # Optionally, you can also clear unused memory (this is the closest thing to memory growth in PyTorch)
        torch.cuda.empty_cache()

/home/a/albertopepe/.local/lib/python3.8/site-packages/torch/cuda/__init__.py:138: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11040). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


In [3]:
gc.collect()
torch.cuda.empty_cache() 
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}', flush = True)
os.environ["CUDA_VISIBLE_DEVICES"] = "0, 1"

Using device: cpu


# Quantum Electrodynamics with Geometric Algebra (WIP)

## Theory overview
Quantum Electrodynamics (QED) describes electrons, positrons (anti-electrons) and photons in a 4-dimensional spacetime with fields defined for all spacetime positions $X$. The 4-dimensional spacetime can be described by the [Spacetime Algebra (STA)](https://en.wikipedia.org/wiki/Spacetime_algebra) with basis vectors $\gamma_0, \gamma_1, \gamma_2, \gamma_3$ and corresponding metric $[1, -1, -1, -1]$. It contains two fields. The electron-positron field is a bispinor-field $\psi(X)$ which in the context of Geometric Algebra (GA) is described by even-grade multivectors of the STA. The photon field $A(X)$ is a vector-field (ie. multivectors of degree 1, one basis for each dimension).

A field configuration, also known as a path, $P(X)$ contains values for the two fields at every spacetime position. Our goal is to calculate the QED action using GA which allows us to use algorithms that solve for field configurations . The action is the negative log-likelihood (NLL) of the field configuration, meaning it is a number which tells how likely a given field configuration is. It is not a probability as it is unnormalized. However even with only the NLL we can use sampling algorithms (eg. [Markov-Chain Monte-Carlo](https://en.wikipedia.org/wiki/Markov_chain_Monte_Carlo), [Variational Inference](https://en.wikipedia.org/wiki/Variational_Bayesian_methods)) to sample field configurations so that the sampled distribution matches the normalized distribution.

The Lagrangian is given in Hestenes' article [Real Dirac Theory](https://www.researchgate.net/publication/266578033_REAL_DIRAC_THEORY) in equation (B.6) as

$\mathcal{L} = \langle \hbar (\nabla \psi(X)) i \gamma_3 \widetilde{\psi}(X) - e A(X) \psi(X) \gamma_0 \widetilde{\psi}(X) - m \psi(X) \widetilde{\psi}(X) \rangle$

where $\langle ... \rangle$ denotes getting the scalar part, $i = \gamma_2 \gamma_1$, $\nabla = \sum_{i=0}^{3} \gamma_i \delta^i$ and $\widetilde{\psi}(X)$ is the grade-reversal of $\psi$.

The action $S(P)$ for a field-configuration $P=(\psi, A)$ is calculated by integrating the Lagrangian $\mathcal{L}(P, X)$ over all space-time positions $X$.

$S(\psi, A) = \int_{X \in \mathcal{X}} \mathcal{L}(\psi, A, X) dx$

Finally as we are doing this numerically we need to discretize spacetime into a 4-dimensional grid. Integrals over spacetime then become sums over the grid. Derivatives become finite-differences or more complicated operations to avoid the [aliasing](https://arxiv.org/abs/hep-lat/0207008) which results in the [fermion doubling](https://en.wikipedia.org/wiki/Fermion_doubling) problem.

## Getting started
Let's start by defining the spacetime algebra as a geometric algebra in 1 time and 3 space dimensions with metric $[1, -1, -1, -1]$.

In [4]:
sta = GeometricAlgebra([1, -1, -1, -1])
for basis in sta.basis_mvs:
    sta.print(basis)

MultiVector[1.00*e_0]
MultiVector[1.00*e_1]
MultiVector[1.00*e_2]
MultiVector[1.00*e_3]


We can see our four basis vectors displayed here each with a different ... basis. Let's try squaring them.

In [5]:
print("e_0^2:", sta(sta.e0) ** 2)
print("e_1^2:", sta(sta.e1) ** 2)
print("e_2^2:", sta(sta.e2) ** 2)
print("e_3^2:", sta(sta.e3) ** 2)

e_0^2: MultiVector[1.00*1]
e_1^2: MultiVector[-1.00*1]
e_2^2: MultiVector[-1.00*1]
e_3^2: MultiVector[-1.00*1]


Squaring the basis vectors gave us back another purely scalar multivector. The squared bases indeed return the correct metric.

We can create new multivectors of different kinds using the geometric algebra `sta_ga` object. Let's create some vectors such as the elements of the photon field and perform some operations to get a feel for them. We can use the methods on `sta_ga`, most of which take a `batch_shape` that says how many elements you want (`[]` meaning just a single one) and a `kind` that describes which elements it will set (eg. `"even"`, `"mv"` (meaning all), `"vector"`, `"scalar"`, ...). Alternatively we can just build everything out of the basis vectors ourselves by adding and multiplying them.

In [6]:
v1 = sta.from_tensor_with_kind(torch.ones(4), kind="vector")
sta.print("v1:", v1)

v2 = sta.basis_mvs[0] + sta.basis_mvs[1]
sta.print("v2:", v2)

sta.print("v1 * v2 (Geometric product):", sta.geom_prod(v1, v2))
sta.print("v1 | v2 (Inner product):", sta.inner_prod(v1, v2))
sta.print("v1 ^ v2 (Exterior product):", sta.ext_prod(v1, v2))

v3 = v1 + v2
sta.print("v3 = v1 + v2:", v3)
sta.print("v1 | v3:", sta.inner_prod(v1, v3))
sta.print("v1 ^ v3:", sta.ext_prod(v1, v3))

v4 = sta.geom_prod(v1, v2)
sta.print("v4 = v1 * v2:", v3)
sta.print("v1^-1 * v4:", sta.geom_prod(sta.inverse(v1), v4), "should be", v2)

v1: MultiVector[1.00*e_0 + 1.00*e_1 + 1.00*e_2 + 1.00*e_3]
v2: MultiVector[1.00*e_0 + 1.00*e_1]
v1 * v2 (Geometric product): MultiVector[-1.00*e_02 + -1.00*e_03 + -1.00*e_12 + -1.00*e_13]
v1 | v2 (Inner product): MultiVector[]
v1 ^ v2 (Exterior product): MultiVector[-1.00*e_02 + -1.00*e_03 + -1.00*e_12 + -1.00*e_13]
v3 = v1 + v2: MultiVector[2.00*e_0 + 2.00*e_1 + 1.00*e_2 + 1.00*e_3]
v1 | v3: MultiVector[-2.00*1]
v1 ^ v3: MultiVector[-1.00*e_02 + -1.00*e_03 + -1.00*e_12 + -1.00*e_13]
v4 = v1 * v2: MultiVector[2.00*e_0 + 2.00*e_1 + 1.00*e_2 + 1.00*e_3]
v1^-1 * v4: MultiVector[1.00*e_0 + 1.00*e_1] should be MultiVector[1.00*e_0 + 1.00*e_1]


Now let's do the same for the bispinors (elements of even degree).

In [7]:
b1 = sta.from_tensor_with_kind(torch.ones(8), kind="even")
sta.print("b1:", b1)

b2 = sta.from_scalar(4.0) + sta.geom_prod(sta.basis_mvs[0], sta.basis_mvs[1]) + sta.geom_prod(sta.basis_mvs[0], sta.basis_mvs[1])
sta.print("b2:", b2)

sta.print("b1 | b2:", sta.inner_prod(b1, b2))
sta.print("b1 ^ b2:", sta.ext_prod(b1, b2))

b3 = sta.geom_prod(b1, b2)
sta.print("b3 = b1 * b2:", b3)
sta.print("b3 * b2^-1:", sta.geom_prod(b3, sta.inverse(b2)), "should be", b1)

sta.print("~b2 (Grade reversal):", sta.reversion(b2))
sta.print("Scalar part of b2:", sta.keep_blades_with_name(b2, ""))
sta.print("e_01 part of b2:", sta.keep_blades_with_name(b2, "01"))

b1: MultiVector[1.00*1 + 1.00*e_01 + 1.00*e_02 + 1.00*e_03 + 1.00*e_12 + 1.00*e_13 + 1.00*e_23 + 1.00*e_0123]
b2: MultiVector[4.00*1 + 2.00*e_01]
b1 | b2: MultiVector[6.00*1 + 6.00*e_01 + 4.00*e_02 + 4.00*e_03 + 4.00*e_12 + 4.00*e_13 + 6.00*e_23 + 4.00*e_0123]
b1 ^ b2: MultiVector[4.00*1 + 6.00*e_01 + 4.00*e_02 + 4.00*e_03 + 4.00*e_12 + 4.00*e_13 + 4.00*e_23 + 6.00*e_0123]
b3 = b1 * b2: MultiVector[6.00*1 + 6.00*e_01 + 6.00*e_02 + 6.00*e_03 + 6.00*e_12 + 6.00*e_13 + 6.00*e_23 + 6.00*e_0123]
b3 * b2^-1: MultiVector[1.00*1 + 1.00*e_01 + 1.00*e_02 + 1.00*e_03 + 1.00*e_12 + 1.00*e_13 + 1.00*e_23 + 1.00*e_0123] should be MultiVector[1.00*1 + 1.00*e_01 + 1.00*e_02 + 1.00*e_03 + 1.00*e_12 + 1.00*e_13 + 1.00*e_23 + 1.00*e_0123]
~b2 (Grade reversal): MultiVector[4.00*1 + -2.00*e_01]
Scalar part of b2: MultiVector[4.00*1]
e_01 part of b2: MultiVector[2.00*e_01]


Now we hopefully have some feel for how to operate with the geometric algebra numbers. So far we only worked with single numbers, but we can define a field (ie. a number for every grid point) by passing in a `batch_shape` that is the size of our grid. When printing the fields we won't see the actual numbers anymore, we will only see which blades are non-zero and the batch shape. However we can still access all of the numbers with the usual indexing rules.

In [8]:
a = sta.from_tensor_with_kind(torch.ones((10, 10, 10, 10, 4)), kind="vector")
sta.print("A(X):", a)

sta.print("A(t=0, x=5, y=3, z=9):", a[0, 5, 3, 9])
sta.print("A(t=0, z=[3,4,5]):", a[0, :, :, 3:6])
sta.print("e_0 part of A(X):", sta.select_blades_with_name(a, "0").shape)
sta.print("A(0, 0, 0, 0) * ~A(0, 0, 0, 0):", sta.geom_prod(a, sta.reversion(a))[0, 0, 0, 0])

A(X): MultiVector[batch_shape=torch.Size([10, 10, 10, 10])]
A(t=0, x=5, y=3, z=9): MultiVector[1.00*e_0 + 1.00*e_1 + 1.00*e_2 + 1.00*e_3]
A(t=0, z=[3,4,5]): MultiVector[batch_shape=torch.Size([10, 10, 3])]


/gpfs/home/a/albertopepe/linesregistration/torchga/torchga.py:500: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  blade_indices = torch.tensor(blade_indices)


e_0 part of A(X): torch.Size([10, 10, 10, 10])
A(0, 0, 0, 0) * ~A(0, 0, 0, 0): MultiVector[-2.00*1]


By now you will probably believe me that we can do the same to create a bispinor field, so instead let's see how we can calculate derivatives.

As mentioned in the beginning, derivatives become finite differences. To calculate finite differences we can take a copy of the field, shift it back by one in a dimension and subtract it. For instance of we were to calculate the derivative
in the time direction we would shift the entire field by -1 along the time axis to get `A(X + TimeDirection * GridSpacing)` and subtract the actual field from this shifted field. All that is left then is to divide by the grid spacing.

`d/dt A(X) = (A(X + TimeDirection * GridSpacing) - A(X)) / GridSpacing`

To actually do the shifting we will use the `with_changes` method which allows copying of the multivector and overriding of its blade values so we will just shift the blade values themselves using [tf.roll](https://www.tensorflow.org/api_docs/python/tf/roll). A better abstraction that doesn't require using the internal blade values might be added later.

In [9]:
def finite_differences(field, axis, spacing):
    # Roll the field tensor along the specified axis by shifting -1 position
    shifted_field = torch.roll(field, shifts=-1, dims=axis)
    
    # Compute the finite difference along the specified axis
    return (shifted_field - field) / spacing

In [10]:
deriv_t_a = finite_differences(a, axis=0, spacing=0.1)
sta.print("d/dt A(X) = (A(X + TimeDirection * GridSpacing) - A(X)) / GridSpacing:", deriv_t_a)
sta.print("d/dt A(0, 0, 0, 0):", deriv_t_a[0, 0, 0, 0])

d/dt A(X) = (A(X + TimeDirection * GridSpacing) - A(X)) / GridSpacing: MultiVector[batch_shape=torch.Size([10, 10, 10, 10])]
d/dt A(0, 0, 0, 0): MultiVector[]


Maybe expectedly, as our field is just a constant value everywhere, we are left with a field that is zero everywhere. Now we have a finite differences operation that will work on fields of any kind.

Now we have all the tools we need to actually calculate the QED action given a field configuration. As a reminder, the QED Lagrangian is given by

$\mathcal{L} = \langle \hbar (\nabla \psi(X)) i \gamma_3 \widetilde{\psi}(X) - e A(X) \psi(X) \gamma_0 \widetilde{\psi}(X) - m \psi(X) \widetilde{\psi}(X) \rangle$

and the action $S(\psi, A)$ is the spacetime integral (now sum) over it.

Let's start with the mass term on the right $m \psi(X) \widetilde{\psi}(X)$.

In [11]:
def get_mass_term(psi, electron_mass):
    return electron_mass * sta.geom_prod(psi, sta.reversion(psi))

In [12]:
# Define psi as some arbitrary even-graded field for now
psi = sta.from_tensor_with_kind(torch.ones([10, 10, 10, 10, 8]), kind="even") + sta.from_tensor_with_kind(torch.ones([10, 10, 10, 10, 1]), kind="scalar")
sta.print("Psi:", psi)
sta.print("Psi at (0, 0, 0, 0):", psi[0, 0, 0, 0])

# The electron mass in planck units (hbar=1, c=1) is actually not 1 but something tiny.
# However we won't bother with it for now.
mass_term = get_mass_term(psi=psi, electron_mass=1.0)
sta.print("Mass term:", mass_term)
sta.print("Mass term at (0, 0, 0, 0):", mass_term[0, 0, 0, 0])

Psi: MultiVector[batch_shape=torch.Size([10, 10, 10, 10])]
Psi at (0, 0, 0, 0): MultiVector[2.00*1 + 1.00*e_01 + 1.00*e_02 + 1.00*e_03 + 1.00*e_12 + 1.00*e_13 + 1.00*e_23 + 1.00*e_0123]
Mass term: MultiVector[batch_shape=torch.Size([10, 10, 10, 10])]
Mass term at (0, 0, 0, 0): MultiVector[3.00*1 + 2.00*e_0123]


Next the interaction term in the center that describes the scattering between the electron-positron field and the photon field $e A(X) \psi(X) \gamma_0 \widetilde{\psi}(X)$.

In [13]:
def get_interaction_term(psi, a, electron_charge):
    return sta.geom_prod(electron_charge * a, sta.geom_prod(psi, sta.geom_prod(sta.e("0"), sta.reversion(psi))))

In [14]:
interaction_term = get_interaction_term(psi=psi, a=a, electron_charge=1.0)
sta.print("Interaction term:", interaction_term)
sta.print("Interaction term at (0, 0, 0, 0):", interaction_term[0, 0, 0, 0])

Interaction term: MultiVector[batch_shape=torch.Size([10, 10, 10, 10])]
Interaction term at (0, 0, 0, 0): MultiVector[25.00*1 + -13.00*e_01 + -13.00*e_02 + -21.00*e_03 + -8.00*e_13 + -8.00*e_23]


And finally the momentum term for which we needed the finite differences $\hbar (\nabla \psi(X)) i \gamma_3 \widetilde{\psi}(X)$.

In [15]:
def get_momentum_term(psi, spacing, hbar):
    # Nabla Psi
    dt_psi = finite_differences(psi, axis=0, spacing=spacing)
    dx_psi = finite_differences(psi, axis=1, spacing=spacing)
    dy_psi = finite_differences(psi, axis=2, spacing=spacing)
    dz_psi = finite_differences(psi, axis=3, spacing=spacing)
    d_psi = dt_psi + dx_psi + dy_psi + dz_psi

    return sta.geom_prod(hbar * d_psi, sta.geom_prod(sta.e("213"), sta.reversion(psi)))

In [16]:
momentum_term = get_momentum_term(psi=psi, spacing=0.1, hbar=1.0)

sta.print("Momentum term:", momentum_term)
sta.print("Momentum term at (0, 0, 0, 0):", momentum_term[0, 0, 0, 0]) # Still zero ;(

Momentum term: MultiVector[batch_shape=torch.Size([10, 10, 10, 10])]
Momentum term at (0, 0, 0, 0): MultiVector[]


Now that we have all the terms, we can add them up, sum over all grid points and take the scalar part to get the action.

In [17]:
def get_action(psi, a, spacing, electron_mass, electron_charge, hbar):
    mass_term = get_mass_term(psi=psi, electron_mass=electron_mass)
    interaction_term = get_interaction_term(psi=psi, a=a, electron_charge=electron_charge)
    momentum_term = get_momentum_term(psi=psi, spacing=spacing, hbar=hbar)

    # Sum terms and get scalar part
    lagrangians = (momentum_term - mass_term - interaction_term)[..., 0]

    # Sum lagrangians (one lagrangian for each spacetime point) over spacetime
    # to get a single value, the action.
    return torch.sum(lagrangians)

In [18]:
action = get_action(psi=psi, a=a, spacing=0.1, electron_mass=1.0, electron_charge=1.0, hbar=1.0)
print("Action:", action)

Action: tensor(-280000.)


Now that we can calculate the action for a given field configuration (ie. values for `psi` and `a` at every grid point) we could use a sampling algorithm
to sample fields and calculate quantities of interest such as the correlation function, vacuum energy and more.

In [25]:
import torch
import pyro
import pyro.infer.mcmc as mcmc
from pyro.infer.mcmc import HMC, MCMC

# Define the joint log probability function
def joint_log_prob(psi_config, a_config):
    # This function must take the parameters `psi_config` and `a_config` and return the log probability (negative action)
    # Ensure that `get_action` and `sta.from_tensor_with_kind` work with PyTorch tensors.
    mv_psi_config = sta.from_tensor_with_kind(psi_config, "even")
    mv_a_config = sta.from_tensor_with_kind(a_config, "vector")

    # Calculate the action (negative log likelihood)
    action = get_action(mv_psi_config, mv_a_config, spacing=1e-7, electron_mass=1e-5,
                        electron_charge=0.0854245, hbar=1.0)
    return -action  # Return negative action as the log probability

# Define the sampling function using Pyro's HMC and MCMC
def sample(initial_state, step_size, num_chains=50):
    # Define the HMC kernel using `potential_fn` (joint_log_prob in this case)
    hmc_kernel = HMC(potential_fn=lambda psi, a: -joint_log_prob(psi, a),
                     step_size=step_size,
                     num_steps=3)  # equivalent to num_leapfrog_steps

    # Define the MCMC sampler with multiple chains and 300 samples
    mcmc_sampler = MCMC(hmc_kernel, num_samples=300, warmup_steps=1000, num_chains=num_chains)
    
    # Ensure initial parameters are provided as a dictionary
    initial_params = {
        "psi": initial_state[0],  # Initial psi_config (bispinor field)
        "a": initial_state[1]     # Initial a_config (vector field)
    }

    # Debug: Check the initial parameter shapes to ensure they're correct
    print(f"Initial psi shape: {initial_params['psi'].shape}")
    print(f"Initial a shape: {initial_params['a'].shape}")

    # Run the MCMC sampler with initial parameters
    mcmc_sampler.run(initial_params)

    # Extract samples from the MCMC sampler
    samples = mcmc_sampler.get_samples()
    psi_samples = samples['psi']
    a_samples = samples['a']
    
    return psi_samples, a_samples

# Example usage
gs = 6  # grid size
num_chains = 50

# Initialize the state tensors for both `psi` and `a` fields (use zeros here for example)
initial_state = [
    torch.zeros((num_chains, gs, gs, gs, gs, 8)),  # Initial psi_config (bispinor field)
    torch.zeros((num_chains, gs, gs, gs, gs, 4))   # Initial a_config (vector field)
]

variable_step_size = 0.001  # Define step size

# Run sampling
psi_samples, a_samples = sample(initial_state, variable_step_size, num_chains=num_chains)

# Print the shapes of the samples for debugging
print(f"Psi Samples shape: {psi_samples.shape}")
print(f"A Samples shape: {a_samples.shape}")


ValueError: Must provide valid initial parameters to begin sampling when using `potential_fn` in HMC/NUTS kernel.

In [ ]:
print(chain_samples[0].shape)
print(chain_samples[1].shape)
print(torch.sum(torch.abs(chain_samples[0][0, 0] - chain_samples[0][1, 0])))
print(torch.sum(torch.abs(chain_samples[0][1, 0] - chain_samples[0][2, 0])))

In [ ]:
import matplotlib.pyplot as plt

fig, axes = plt.subplots(2, 2, sharex=True, sharey=True, figsize=(5, 5))
for i in range(4):
    ax = axes[i % 2][i // 2]
    im = ax.imshow(chain_samples[1][0, 0, 0, 0, :, :, i])
    fig.colorbar(im, ax=ax)
fig.show()

In [ ]:
fig, axes = plt.subplots(2, 4, sharex=True, sharey=True, figsize=(10, 5))
for i in range(8):
    ax = axes[i % 2][i // 2]
    im = ax.imshow(chain_samples[0][0, 0, 0, 0, :, :, i])
    fig.colorbar(im, ax=ax)
fig.show()

fig, axes = plt.subplots(2, 4, sharex=True, sharey=True, figsize=(10, 5))
for i in range(8):
    ax = axes[i % 2][i // 2]
    im = ax.imshow(chain_samples[0][0, 0, 0, 1, :, :, i])
    fig.colorbar(im, ax=ax)
fig.show()

fig, axes = plt.subplots(2, 4, sharex=True, sharey=True, figsize=(10, 5))
for i in range(8):
    ax = axes[i % 2][i // 2]
    im = ax.imshow(chain_samples[0][0, 0, 0, 2, :, :, i])
    fig.colorbar(im, ax=ax)
fig.show()

In [ ]:
fig, axes = plt.subplots(2, 4, sharex=True, sharey=True, figsize=(10, 5))
for i in range(8):
    ax = axes[i % 2][i // 2]
    im = ax.imshow(chain_samples[0][0, 0, 0, 0, :, :, i])
    fig.colorbar(im, ax=ax)
fig.show()

fig, axes = plt.subplots(2, 4, sharex=True, sharey=True, figsize=(10, 5))
for i in range(8):
    ax = axes[i % 2][i // 2]
    im = ax.imshow(chain_samples[0][100, 0, 0, 0, :, :, i])
    fig.colorbar(im, ax=ax)
fig.show()

fig, axes = plt.subplots(2, 4, sharex=True, sharey=True, figsize=(10, 5))
for i in range(8):
    ax = axes[i % 2][i // 2]
    im = ax.imshow(chain_samples[0][200, 0, 0, 0, :, :, i])
    fig.colorbar(im, ax=ax)
fig.show()

In [ ]:
with plt.style.context("bmh"):
    def plot_correlations(ax, samples, axis):
        correlation_by_shift = []
        correlation_std_by_shift = []
        shifts = list(range(1, samples.shape[axis]))

        #if samples.shape[-1] == 8:
        #    samples = sta.from_tensor_with_kind(samples, "even")
        #elif samples.shape[-1] == 4:
        #    samples = sta.from_tensor_with_kind(samples, "vector")

        for i in shifts:
            shifted = torch.roll(samples, shift=-i, dims=axis)
            correlations = torch.mean(samples * shifted, axis=[-1, -2, -3, -4, -5])
            #correlations = tf.reduce_mean(sta.inner_prod(samples, shifted), axis=[-1, -2, -3, -4, -5])
            correlation_by_shift.append(torch.mean(correlations))
            correlation_std_by_shift.append(torch.std(correlations))
        ax.errorbar(shifts, correlation_by_shift, correlation_std_by_shift, capsize=5)

    fig, axes = plt.subplots(4, sharex=True, sharey=True, figsize=(14, 8))
    plot_correlations(axes[0], chain_samples[0], axis=-2)
    plot_correlations(axes[1], chain_samples[0], axis=-3)
    plot_correlations(axes[2], chain_samples[0], axis=-4)
    plot_correlations(axes[3], chain_samples[0], axis=-5)

    fig, axes = plt.subplots(4, sharex=True, sharey=True, figsize=(14, 8))
    plot_correlations(axes[0], chain_samples[1], axis=-2)
    plot_correlations(axes[1], chain_samples[1], axis=-3)
    plot_correlations(axes[2], chain_samples[1], axis=-4)
    plot_correlations(axes[3], chain_samples[1], axis=-5)